In [ ]:
#####################################################################
#########   MAIN CODE FOR CALLING CSV FILESS TO DOWNLOAD    #########
#####################################################################
 
import datetime
import subprocess
import os
import time

def wait_until(target_time):
    """Wait until the specified target time."""
    now = datetime.datetime.now()
    target_time = now.replace(hour=target_time.hour, minute=target_time.minute, second=0, microsecond=0)

    # If the target time is earlier today, schedule for tomorrow
    if target_time < now:
        target_time += datetime.timedelta(days=1)

    sleep_duration = (target_time - now).total_seconds()
    print(f"Waiting for {sleep_duration / 60:.2f} minutes until {target_time}.")
    time.sleep(sleep_duration)

def run_python_script(script_path):
    """Run a Python script using subprocess."""
    try:
        print(f"Running script: {script_path}")
        result = subprocess.run(["python", script_path], check=True, text=True, capture_output=True)
        print(f"Output of {script_path}: {result.stdout}")
        if result.stderr:
            print(f"Error output of {script_path}: {result.stderr}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running {script_path}: {e}")

def clear_csv_files(directory):
    """Delete all CSV files in the specified directory."""
    print(f"Clearing CSV files in directory: {directory}")
    for file_name in os.listdir(directory):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory, file_name)
            try:
                os.remove(file_path)
                print(f"Deleted {file_path}")
            except Exception as e:
                print(f"Failed to delete {file_path}: {e}")

def check_file_exists(file_path):
    """Check if the file exists."""
    if not os.path.isfile(file_path):
        print(f"File not found: {file_path}")
        return False
    print(f"File exists: {file_path}")
    return True

def main():
    # Define the path to script1
    script1 = "D:\\AA_trading_Algos\\modified_ALgo\\downloadCSV.py"
    downloads_folder = "C:\\Users\\omkar\\Downloads"

    # Define the execution window for script1
    script1_start_time = datetime.time(10, 14)  # Start time for script1
    script1_end_time = datetime.time(11, 44)  # End time for script1
    interval_minutes = 15  # Interval to run script1

    while True:
        now = datetime.datetime.now().time()

        # Run script1 within the specified time window at regular intervals
        if script1_start_time <= now <= script1_end_time:
            # Check if script1 exists
            if not check_file_exists(script1):
                print("Script1 is missing. Exiting...")
                return

            # Run script1 at the defined intervals
            print(f"Running {script1}...")
            run_python_script(script1)
            print(f"Completed running {script1}")

            # Wait for the next interval
            next_interval_time = datetime.datetime.combine(datetime.date.today(), now) + datetime.timedelta(minutes=interval_minutes)
            next_interval_time = next_interval_time.time()
            print(f"Waiting for the next interval ({interval_minutes} minutes)...")
            wait_until(next_interval_time)
        else:
            print("Outside of the script1 execution window. Waiting until the next execution window...")
            # Calculate the time to sleep until the next execution window
            next_execution_time = datetime.datetime.combine(datetime.date.today(), script1_start_time)
            if now > script1_end_time:
                # If the current time is past the end time, schedule for the next day
                next_execution_time += datetime.timedelta(days=1)
            time_to_sleep = (next_execution_time - datetime.datetime.now()).total_seconds()
            time.sleep(time_to_sleep)

    # Clear CSV files in the Downloads folder
    print("Clearing CSV files in Downloads folder...")
    clear_csv_files(downloads_folder)
    print("Completed clearing CSV files.")

    print("Main function completed.")

if __name__ == "__main__":
    main()


In [ ]:
import csv
from datetime import datetime as dt_datetime, timedelta
import time
import threading
import logging
import pandas as pd
from NorenRestApiPy.NorenApi import NorenApi
import pyotp
import time
from datetime import datetime, timedelta

class ShoonyaApiPy(NorenApi):
    def __init__(self):
        super().__init__(host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/')

# Configuration Constants
USER = 'FA74468'
PWD = 'GURU222kore$'
TOKEN = 'XT2L66VT73Q22P33BNCHKN6WA2Q37KK6'
VC = 'FA74468_U'
APP_KEY = 'c98e82a190da8181c80fb94cf0a31144'
IMEI = 'abc1234'
CSV_FILE_PATH = "C:\\Users\\omkar\\Downloads\\Backtest BB Blast_Omk, Technical Analysis Scanner.csv"
REMOVE_STOCKS = ['M&M-EQ', 'M&MFIN-EQ', 'J&KBANK-EQ']
processed_stocks = set()  # Using a set to avoid duplicates

# Logging Configuration
logging.basicConfig(
    filename='D:\\AA_trading_Algos\\modified_ALgo\\trading_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w'
)

# Global Variables
stocksList = []
slArray = []
tgtArray = []
stock_names = []
rpnl_values = []
quantities = []

# Initialize API
api = ShoonyaApiPy()
factor2 = pyotp.TOTP(TOKEN).now()
api.login(userid=USER, password=PWD, twoFA=factor2, vendor_code=VC, api_secret=APP_KEY, imei=IMEI)

def place_buy_orders_based_on_positions():
    global stock_names, rpnl_values, quantities, processed_stocks
    logging.debug("Running place_buy_orders_based_on_positions")
 
    stocks_to_buy = []  # List to keep track of stocks meeting conditions
    positions = api.get_positions()
    df = pd.DataFrame(positions)
    if 'tsym' not in df.columns or 'rpnl' not in df.columns or 'daysellqty' not in df.columns:
        logging.error("No positions data found or data is not in the expected format.")
        return
 
    stock_names = df['tsym'].dropna().tolist()
    rpnl_values = pd.to_numeric(df['urmtom'], errors='coerce').fillna(float('nan')).tolist()
    quantities = pd.to_numeric(df['daysellqty'], errors='coerce').fillna(0).astype(int).tolist()

    for i in range(len(stock_names)):
        print(stock_names[i] )
        print(rpnl_values[i])
        # Check the PnL conditions and make sure the stock isn't already processed
        if rpnl_values[i] is not None and (rpnl_values[i] <= -40 or rpnl_values[i] >= 79) and stock_names[i] not in processed_stocks:
            print(f"Placing order for: {stock_names[i]}")
            try:
                ret = api.place_order(
                    buy_or_sell='B',
                    product_type='I',
                    exchange='NSE',
                    tradingsymbol=stock_names[i],
                    quantity=quantities[i],
                    discloseqty=0,
                    price_type='MKT',
                    retention='DAY',
                    remarks='my_order_001'
                )
                # Add to processed_stocks only if the conditions are met and the order is placed
                processed_stocks.add(stock_names[i])
                stocks_to_buy.append(stock_names[i])  # Add to the list of stocks that met the condition
                logging.info(f"Buy order placed for {stock_names[i]}. Quantity: {quantities[i]}, PnL: {rpnl_values[i]}")
            except Exception as e:
                logging.error(f"Error placing buy order for {stock_names[i]}: {e}")
 
    if not stocks_to_buy:
        logging.info("No stocks met the conditions for buying.")
    else:
        logging.info(f"Stocks meeting buy conditions: {', '.join(stocks_to_buy)}")
 
    logging.info("place_buy_orders_based_on_positions function completed.")

print(processed_stocks)

# Function to be executed
def scheduled_run():
    place_buy_orders_based_on_positions()
    print(processed_stocks)

# Get the current time
start_time = datetime.now()

# Set the duration for which the script should run (10 minutes)
end_time = start_time + timedelta(minutes=10)

# Run the loop until the end time is reached
while datetime.now() < end_time:
    scheduled_run()
    # Wait for 10 seconds before running again
    time.sleep(5)          

print("Execution completed.")
